In [1]:
!pip install selenium


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime

In [18]:
options = Options()
options.add_argument("--headless")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1425,1004")
options.page_load_strategy = "none"

driver = webdriver.Chrome(options=options)

# Load the investopedia news page. 
ticker = "AAPL"
driver.get(f"https://ih.advfn.com/p.php?pid=news&symbol=N%5E{ticker}")

time.sleep(1)

df_news_list = []

loop_count = 0
while True:

    try: 
        # Waits for table element with ID newsList to exist, gets table row elements
        news_table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'newsList')))
        table_rows = news_table.find_element(By.TAG_NAME, "table").find_elements(By.TAG_NAME, "tr")

        # Loop through table rows, initialise dataframe on first row and then add data
        for i, row in enumerate(table_rows):
            row_data = [td.text.strip() for td in row.find_elements(By.TAG_NAME, "td")]

            if i == 0:
                df = pd.DataFrame(columns=row_data)
            else:
                df.loc[len(df.index)] = row_data

        df_news_list.append(df)

        # Wait for next button to exist
        next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, 'Next >>')))

    except Exception as e:
        print(e)
        print("Couldn't find table of news. Refreshing page and trying again")
        driver.refresh()
        time.sleep(2)
        continue

    else:
        # Gets date time object from the date of the last article added to the dataframe
        last_date = datetime.strptime(df.iloc[-1, 0], "%m/%d/%Y")
        
        #### BREAK POINT
        # Break if last article date is earlier than 2019
        if(last_date < datetime(2019, 1, 1)):
            break

        # Click next button to load next page
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        next_button.click()

        time.sleep(1)

        loop_count += 1
        print(f"{loop_count} pages of articles added to dataframe, back to {last_date}")

df_news = pd.concat(df_news_list)

driver.close()
driver.quit()

1 pages of articles added to dataframe, back to 2024-01-29 00:00:00
2 pages of articles added to dataframe, back to 2024-01-10 00:00:00
3 pages of articles added to dataframe, back to 2023-11-29 00:00:00


KeyboardInterrupt: 

In [16]:
df_news

,Date,Time,Source,Headline,Symbol,Company
0,02/16/2024,6:08AM,IH Market News,"Nike Workforce Reduction, The Trade Desk Soars...",NASDAQ:AAPL,Apple Inc
1,02/15/2024,6:34AM,IH Market News,Cisco Systems Stock Drops 5% Following Restruc...,NASDAQ:AAPL,Apple Inc
2,02/14/2024,4:47PM,Edgar (US Regulatory),Form SC 13G/A - Statement of acquisition of be...,NASDAQ:AAPL,Apple Inc
3,02/12/2024,4:17PM,Edgar (US Regulatory),Form SC 13G/A - Statement of acquisition of be...,NASDAQ:AAPL,Apple Inc
4,02/09/2024,4:26PM,Edgar (US Regulatory),Form PX14A6G - Notice of exempt solicitation s...,NASDAQ:AAPL,Apple Inc
...,...,...,...,...,...,...
25,01/02/2019,5:03PM,Dow Jones News,Apple Revises Guidance,NASDAQ:AAPL,Apple Inc
26,01/02/2019,4:32PM,Edgar (US Regulatory),Current Report Filing (8-k),NASDAQ:AAPL,Apple Inc
27,01/02/2019,4:32PM,Business Wire,Letter from Tim Cook to Apple Investors,NASDAQ:AAPL,Apple Inc
28,12/28/2018,3:02AM,Dow Jones News,Buybacks Come Back to Bite Firms -- WSJ,NASDAQ:AAPL,Apple Inc


In [17]:
# SAVE TO CSV
df_news.to_csv("investopedia_articles.csv", index=False)